<a href="https://colab.research.google.com/github/amangupta2/ai_weather/blob/main/Handling_Missing_Values_from_Metrostat_(using_darts).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Note that pip might complain about some versions but the notebook should still work as expected.
#!pip install git+https://github.com/google-research/weatherbenchX.git
#!pip install stationbench
#!pip install cartopy
#! pip install netCDF4
! pip install meteostat
! pip install MetPy
! pip install darts
# install python library: regionalmask if needed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.7/169.7 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:0

In [ ]:
#import matplotlib.pyplot as plt
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature

#from metpy.calc import specific_humidity_from_dewpoint, relative_humidity_from_specific_humidity
#from metpy.units import units

#
#from netCDF4 import Dataset

#import numpy as np
from datetime import datetime, timedelta
#import matplotlib.pyplot as plt
from meteostat import Point, Daily, Monthly, Hourly, Stations
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature

# https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.relative_humidity_from_specific_humidity.html
# https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.specific_humidity_from_dewpoint.html
# https://confluence.ecmwf.int/pages/viewpage.action?pageId=133262398
from metpy.calc import relative_humidity_from_specific_humidity
from metpy.units import units

from darts.utils.missing_values import fill_missing_values
import pandas as pd
from darts import TimeSeries
from darts.dataprocessing.transformers import MissingValuesFiller

In [ ]:
stations=Stations()
stations = stations.region('IN')
stations_data = stations.fetch()
#stations_name = stations_data.name

# delhi
lat0=28.6439835
lon0=76.9280785

#madras
#lat0=13.
#lon0=80.1833

# calcutta
#lat0=22.5333
#lon0=88.3333


# find stations nearby a given lat lon
A = stations.nearby(lat0,lon0, radius=25000) # 25 or 50 km could be a good choice for a 25 km grid - there will be repetitions, but too far a radar would lead to erroneous predictions for clouds and sunlight.
A.fetch()

# station availability around a certain data
A = stations.inventory('hourly',datetime(2020,1,1))
A.fetch()
# to read a particular station from a table
A.iloc[-2]

delhi       = Point(28.6439835,76.9280785)
radar_data  = Hourly(delhi,start,end)

In [ ]:
ix=0
stat  = stations.fetch().iloc[ix]
point = Point(stat.latitude, stat.longitude )
start = stat.hourly_start
end   = stat.hourly_end
# get a table of all variables recorded over stat from start to end
radar_data  = Hourly(point,start,end)
# to see the data
radar_data.fetch()

series = TimeSeries.from_dataframe(radar_data.fetch(), time_col=0, value_cols="temp")
filler = MissingValuesFiller()
filled_series = filler.transform(series)
filled_series
radar_data_filled = filled_series.to_dataframe()

In [ ]:
#radar_data.fetch()['temp']

fig=plt.figure(figsize=(50, 8), dpi= 80, facecolor='w', edgecolor='k')
plt.plot(radar_data_filled,'o',markersize=1)
plt.plot(radar_data.fetch()['temp'])
#plt.plot(filled,'o',markersize=1)
